<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_for_knowledgeGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="gemma3:12b" # "llama3.2"
model_2_id="deepseek-r1:8b"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install openai --quiet

In [ ]:
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {model_2_id}")

In [ ]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [ ]:
def call(system_prompt: str, message: str, model: str = modelid ) -> str:
  completion = llm.chat.completions.create(
      model=modelid,
      messages=[
          {"role": "system", "content": system_prompt},
          {
              "role": "user",
              "content": message,
          },
      ],
  )

  return completion.choices[0].message.content

In [ ]:
def marketing_campaign_agent(product : str):
  """

     Your task is to create an compelling marketing campaign plan for the product.

    ## Product Description
    {product}

     ## Marketing Campaign Plan:L
     Your marketing campaign plan:

  """
  return call ("your are a marketing campaign export.", marketing_campaign_agent.__doc__.format(product=product) )

In [ ]:
def ready_validation_agent(marketing_text : str) -> bool:
  """

  Attached is a product descipription of a new product we are launching.

  Also attached is a marketing campaign plan for the product launch.

  Your task is to read the marketing campaign plan and assess wether this is good enough to get finalized or wether it would benefit from another revision.

  Answer with TRUE or FALSE. add nothing else to your response.

  ## Product Description
  {product}

  ## Marketing Text
  {text}

  ## Your Assessment
  Your Assessment:

    """
  result = call ("your are a marketing campaign export.", ready_validation_agent.__doc__.format(product=product, text=marketing_text) , model=model_2_id)
  print (result)
  return result.strip().lower() == "true"

In [ ]:
def refinements_agent(product : str, marketing_text : str) -> str:
  """

  Attached is a product descipription of a new product we are launching.

  Also attached is a draft marketing campaign plan for the product launch.

  Provide a critique of this draft marketing plan in preparation of the final version.

  ## Product Description
  {product}

  ## Marketing Text
  {text}

  ## Critique
  Your critique:

  """
  return call ("your are a marketing campaign export.", refinements_agent.__doc__.format(product=product, text=marketing_text) )

In [ ]:
def refine_agent(product : str, marketing_text : str, refinements : str ) -> str:
  """

  Attached are 3 documents
    - a product descipription of a new product we are launching.
    - a draft marketing campaign plan for the product launch.
    - a critique of this draft marketing plan in preparation of the final version

  Please write another version of the marketing campaign plan taking into account the feedback in the critique.

  ## Product Description
  {product}

  ## Marketing Text
  {text}

  ## Critique
  {refinements}

  ## Final Marketing Text
  Your final version of the marketing campaign plan:


  """
  return call ("your are a marketing campaign export.", refine_agent.__doc__.format(product=product, text=marketing_text, refinements=refinements) )

In [ ]:
product = "iPhone 15"

iteration=1
marketing_text = marketing_campaign_agent(product)
while not ready_validation_agent(marketing_text) and iteration<5:
  print (f"ITERATION: {iteration}" )
  refinements = refinements_agent(product, marketing_text)
  next_text_iteration = refine_agent(product, marketing_text, refinements)
  print (refinements)
  iteration+=1
  marketing_text = next_text_iteration


TypeError: call() got an unexpected keyword argument 'modelid'

In [ ]:
print (next_text_iteration)


In [ ]:
print (marketing_text)
print ("###### critique")
print (refinements)
print ("###### next text iteration")
print (next_text_iteration)